In [23]:
import numpy as np
import pandas as pd
import zipfile
import matplotlib.pyplot as plt

In [24]:
# Load the npz file
with np.load('capstone_sbb.npz', allow_pickle=False) as npz_file:
    # It's a dictionary-like object
    print(list(npz_file.keys()))

['df_merged_train', 'df_merged_train_columns', 'df_merged_train_dep', 'df_merged_train_dep_columns', 'df_merged_train_arr', 'df_merged_train_arr_columns', 'df_merged', 'df_merged_columns', 'df_clean_final', 'df_clean_final_columns', 'df_stops_clean', 'df_stops_clean_columns']


In [25]:
with np.load('capstone_sbb.npz', allow_pickle=True) as npz_file:
    # Load the arrays
    df_merged_zug_x = npz_file['df_merged_train']
    df_merged_zug_columns_x = npz_file['df_merged_train_columns']
    
    df_merged_zug_dep_x = npz_file['df_merged_train_dep']
    df_merged_zug_dep_columns_x = npz_file['df_merged_train_dep_columns']
    df_merged_zug_arr_x = npz_file['df_merged_train_arr']
    df_merged_zug_arr_columns_x = npz_file['df_merged_train_arr_columns']
    
    df_merged_x = npz_file['df_merged']
    df_merged_columns_x = npz_file['df_merged_columns']
    df_clean_final_x = npz_file['df_clean_final']
    df_clean_final_columns_x = npz_file['df_clean_final_columns']
    df_stops_clean_x = npz_file['df_stops_clean']
    df_stops_clean_columns_x = npz_file['df_stops_clean_columns']


print('df_merged_final:', df_merged_zug_x.shape)
print('df_merged_final_columns:', df_merged_zug_columns_x.shape)
print('df_merged_final_departure:', df_merged_zug_dep_x.shape)
print('df_merged_final_departure_columns:', df_merged_zug_dep_columns_x.shape)
print('df_merged_final_arrival:', df_merged_zug_arr_x.shape)
print('df_merged_final_arrival_columns:', df_merged_zug_arr_columns_x.shape)
print('df_merged:', df_merged_x.shape)
print('df_merged_columns:', df_merged_columns_x.shape)
print('df_clean_final:', df_clean_final_x.shape)
print('df_clean_final_columns:', df_clean_final_columns_x.shape)
print('df_stops_clean:', df_stops_clean_x.shape)
print('df_stops_clean_columns:', df_stops_clean_columns_x.shape)

df_merged_final: (158121, 39)
df_merged_final_columns: (39,)
df_merged_final_departure: (144111, 39)
df_merged_final_departure_columns: (39,)
df_merged_final_arrival: (144117, 39)
df_merged_final_arrival_columns: (39,)
df_merged: (1854124, 38)
df_merged_columns: (38,)
df_clean_final: (1854124, 27)
df_clean_final_columns: (27,)
df_stops_clean: (27738, 10)
df_stops_clean_columns: (10,)


In [26]:
df_merged_zug = pd.DataFrame(df_merged_zug_x, columns=df_merged_zug_columns_x)
df_merged_zug_dep = pd.DataFrame(df_merged_zug_dep_x, columns=df_merged_zug_dep_columns_x)
df_merged_zug_arr = pd.DataFrame(df_merged_zug_arr_x, columns=df_merged_zug_arr_columns_x)
df_merged = pd.DataFrame(df_merged_x, columns=df_merged_columns_x)
df_clean_final = pd.DataFrame(df_clean_final_x, columns=df_clean_final_columns_x)
df_stops_clean = pd.DataFrame(df_stops_clean_x, columns=df_stops_clean_columns_x)

In [27]:
df_merged_zug.nunique()

operating_day                                 1
operating_day_of_week                         1
ride_id                                   14884
operator_short                               44
operator_name                                46
transport_mode                                1
line_id                                   12268
line_name                                    93
deviation_id                                  0
product_id                                   17
add_ride_flag                                 2
missing_ride_flag                             2
stop_id                                    1667
stop_name                                  1667
arrival_time                               1390
eta                                       58452
eta_status_clean                              4
arrival_time_of_day                           8
arrival_delay_min                            84
arrival_delay_bucket                          6
departure_time                          

### Filtering-out features not necessary for Input and Output of models

* **Remove variable which have already been bucketed**
    * arrival_time
    * departure_time
    * operating_day (replaced by 'operating_day_of_week')
<br>
* **Remove duplicate varibales**
    * operator_name (keep operator_short)
    * stop_name (keep stop_id)
<br>
* **Remove irrelavant varibles for classification taks**
    * deviation_id
    * transport_mode (as we focus on 'Zug' only)
    * missing_ride_flag and add_ride_flag
        * missing_ride_flag have been removed as considered as information potentially only available on last-minute - per-default (rides classified as 'important_delay')
        * add_ride_flag have been kept as can be a relevant information to assess delay and probably planned ahead
<br>
* **Remove categorical variable with too high cardinality**
    * ride_id
    * line_id
    * *(thinking of replacing Stop_id by city but decide to keep both for now)*
<br>
* **Remove variables which have been used to build target (and shouldn't be used in test set)**
    * eta
    * eta_status_clean
    * arrival_delay_min
    * etd
    * etd_status_clean
    * departure_delay_min

In [28]:
keep_input = ['operating_day_of_week','operator_short','line_name','product_id','stop_id','arrival_time_of_day','departure_time_of_day','start_middle_end','provider_short','city','canton','longitude_ch','latitude_ch','altitude','transport_mode_stop']
keep_input_cat = ['operating_day_of_week','operator_short','line_name','product_id','stop_id','arrival_time_of_day','departure_time_of_day','start_middle_end','provider_short','city','canton','transport_mode_stop']
keep_output = ['arrival_delay_bucket_final','departure_delay_bucket_final','arrival_important_delay_bucket_final','departure_important_delay_bucket_final']

In [29]:
df_merged_zug[keep_input].nunique()
# thinking of removing stop_id, longitude_ch and latitude_ch

operating_day_of_week       1
operator_short             44
line_name                  93
product_id                 17
stop_id                  1667
arrival_time_of_day         8
departure_time_of_day       8
start_middle_end            3
provider_short             33
city                      877
canton                     26
longitude_ch             1637
latitude_ch              1634
altitude                  681
transport_mode_stop         8
dtype: int64

In [30]:
df_merged_zug[keep_output].shape

(158121, 4)

### Variable transformation (one-hot encoding)

In [31]:
pd.get_dummies(df_merged_zug[keep_input], columns=keep_input_cat).shape
# decide not to have k-1 columns per variable as I might be working with decision trees which need all columns. 
# would be fine using only k-1 columns for regression, SVMs or CNNs

(158121, 2788)

In [32]:
pd.get_dummies(df_merged_zug[keep_input], columns=keep_input_cat).head()

,longitude_ch,latitude_ch,altitude,operating_day_of_week_Thursday,operator_short_AB-ab,operator_short_ASM-bti,operator_short_ASM-rvo,operator_short_ASM-snb,operator_short_AVA-bd,operator_short_AVA-wsb,...,canton_ZG,canton_ZH,transport_mode_stop_Unbekannt,transport_mode_stop_Zahnradbahn,transport_mode_stop_Zug,transport_mode_stop_Zug_Bus,transport_mode_stop_Zug_Bus_Tram,transport_mode_stop_Zug_Kabinenbahn,transport_mode_stop_Zug_Metro,transport_mode_stop_Zug_Tram
0,2.61266e+06,1.26852e+06,263,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2.61266e+06,1.26852e+06,263,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2.61266e+06,1.26852e+06,263,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,2.61266e+06,1.26852e+06,263,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,2.61266e+06,1.26852e+06,263,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


### Import training, validation and test data sets

In [33]:
training_days = []
for tr in np.arange(1,15):
    #print(str(tr).zfill(2))
    training_days.append(str(tr).zfill(2))
validation_days = []
for va in np.arange(15,22):
    #print(str(va).zfill(2))
    validation_days.append(str(va).zfill(2))
test_days = []
for te in np.arange(22,29):
    #print(str(te).zfill(2))
    test_days.append(str(te).zfill(2))

In [34]:
# training
d = 0
zf = zipfile.ZipFile('20_10.zip') 
for x in training_days:
    temp = pd.read_csv(zf.open('20_10/2020-10-' + x + '_istdaten.csv'),delimiter=';',low_memory=False) # import data for single day (1st October) for 
    print(temp.shape)
    temp_2 = temp[temp['PRODUKT_ID'] == 'Zug']
    print(temp_2.shape)
    d = d + 1
    if d == 1:
        df_train = temp_2
    else: 
        df_train = pd.concat([df_train,temp_2])
    print('Shape final:',df_train.shape)
    print(d)

(1854124, 21)
(157983, 21)
Shape final: (157983, 21)
1
(1873590, 21)
(154120, 21)
Shape final: (312103, 21)
2
(1511048, 21)
(147614, 21)
Shape final: (459717, 21)
3
(1138020, 21)
(138283, 21)
Shape final: (598000, 21)
4
(1863830, 21)
(156951, 21)
Shape final: (754951, 21)
5
(1860703, 21)
(157711, 21)
Shape final: (912662, 21)
6
(1852875, 21)
(157790, 21)
Shape final: (1070452, 21)
7
(1864414, 21)
(157690, 21)
Shape final: (1228142, 21)
8
(1885718, 21)
(158600, 21)
Shape final: (1386742, 21)
9
(1502261, 21)
(146548, 21)
Shape final: (1533290, 21)
10
(1127607, 21)
(136232, 21)
Shape final: (1669522, 21)
11
(1859314, 21)
(156781, 21)
Shape final: (1826303, 21)
12
(1855882, 21)
(156385, 21)
Shape final: (1982688, 21)
13
(1867644, 21)
(155639, 21)
Shape final: (2138327, 21)
14


In [35]:
# validation
d_va = 0
zf = zipfile.ZipFile('20_10.zip') 
for x in validation_days:
    temp_va = pd.read_csv(zf.open('20_10/2020-10-' + x + '_istdaten.csv'),delimiter=';',low_memory=False) # import data for single day (1st October) for 
    print(temp_va.shape)
    temp_va_2 = temp_va[temp_va['PRODUKT_ID'] == 'Zug']
    print(temp_va_2.shape)
    d_va = d_va + 1
    if d_va == 1:
        df_valid = temp_va_2
    else: df_valid = pd.concat([df_valid,temp_va_2])
    print('Shape final:',df_valid.shape)
    print(d_va)

(1859473, 21)
(156386, 21)
Shape final: (156386, 21)
1
(1886155, 21)
(157366, 21)
Shape final: (313752, 21)
2
(1506195, 21)
(147185, 21)
Shape final: (460937, 21)
3
(1129869, 21)
(138045, 21)
Shape final: (598982, 21)
4
(1829842, 21)
(157087, 21)
Shape final: (756069, 21)
5
(1833200, 21)
(156946, 21)
Shape final: (913015, 21)
6
(1823739, 21)
(156784, 21)
Shape final: (1069799, 21)
7


In [36]:
# test
d_te = 0
zf = zipfile.ZipFile('20_10.zip') 
for x in test_days:
    temp_te = pd.read_csv(zf.open('20_10/2020-10-' + x + '_istdaten.csv'),delimiter=';',low_memory=False) # import data for single day (1st October) for 
    print(temp_te.shape)
    temp_te_2 = temp_te[temp_te['PRODUKT_ID'] == 'Zug']
    print(temp_te_2.shape)
    d_te = d_te + 1
    if d_te == 1:
        df_test = temp_te_2
    else: df_test = pd.concat([df_test,temp_te_2])
    print('Shape final:',df_test.shape)
    print(d_te)

(1827837, 21)
(156833, 21)
Shape final: (156833, 21)
1
(1857399, 21)
(158581, 21)
Shape final: (315414, 21)
2
(1475704, 21)
(148250, 21)
Shape final: (463664, 21)
3
(1118470, 21)
(138054, 21)
Shape final: (601718, 21)
4
(1852126, 21)
(157393, 21)
Shape final: (759111, 21)
5
(1860713, 21)
(157416, 21)
Shape final: (916527, 21)
6
(1874026, 21)
(157448, 21)
Shape final: (1073975, 21)
7


In [37]:
df_train.shape

(2138327, 21)

In [38]:
df_valid.shape

(1069799, 21)

In [39]:
df_test.shape

(1073975, 21)

### Cleaning and transforming data as per EDA <a name='index'/>
1. **Renaming columns** - <a href=#rename>here</a>
2. **Update data types** - <a href=#dtypes>here</a>
3. **Handling Null values** - <a href=#null>here</a>
<br>    a. Stops name
<br>    b. Product id
4. **Feature engineering** - <a href=#engineer>here</a>
<br>    a. Ride sequence (start_middle_end flag)
<br>    b. ETA/ETD
<br>    c. Time of day
<br>    d. Delays
5. **Final data sets** - <a href=#final>here</a>
<br>    a. filtering-in only relevant features
<br>    b. merging both data sets
<br>    c. splitting for departure and arrival delays predictions
6. **Data transformation to feed ML models** - <a href=#trans>here</a>
<br>    a. Input features (categorical, numerical, merged)
<br>    b. Output features
7. **Saving data** - <a href=#save>here</a>

In [40]:
df_train['BETRIEBSTAG'].unique()

array(['01.10.2020', '02.10.2020', '03.10.2020', '04.10.2020',
       '05.10.2020', '06.10.2020', '07.10.2020', '08.10.2020',
       '09.10.2020', '10.10.2020', '11.10.2020', '12.10.2020',
       '13.10.2020', '14.10.2020'], dtype=object)

**1) translating columns naming from german to english** <a name='rename'/> | <a href=#index>Back to top</a>

In [41]:
## Training
df_train.rename(columns={'BETRIEBSTAG':'operating_day','FAHRT_BEZEICHNER':'ride_id','BETREIBER_ID':'operator_id','BETREIBER_ABK':'operator_short','BETREIBER_NAME':'operator_name',\
                   'PRODUKT_ID':'transport_mode','LINIEN_ID':'line_id','LINIEN_TEXT':'line_name','UMLAUF_ID':'deviation_id','VERKEHRSMITTEL_TEXT':'product_id','ZUSATZFAHRT_TF':'add_ride_flag',\
                  'FAELLT_AUS_TF':'missing_ride_flag','BPUIC':'stop_id','HALTESTELLEN_NAME':'stop_name','ANKUNFTSZEIT':'arrival_time','AN_PROGNOSE':'eta','AN_PROGNOSE_STATUS':'eta_status'\
                   ,'ABFAHRTSZEIT':'departure_time','AB_PROGNOSE':'etd','AB_PROGNOSE_STATUS':'etd_status','DURCHFAHRT_TF':'ride_through_flag'}, inplace=True)
print(df_train.columns)

## Validation
df_valid.rename(columns={'BETRIEBSTAG':'operating_day','FAHRT_BEZEICHNER':'ride_id','BETREIBER_ID':'operator_id','BETREIBER_ABK':'operator_short','BETREIBER_NAME':'operator_name',\
                   'PRODUKT_ID':'transport_mode','LINIEN_ID':'line_id','LINIEN_TEXT':'line_name','UMLAUF_ID':'deviation_id','VERKEHRSMITTEL_TEXT':'product_id','ZUSATZFAHRT_TF':'add_ride_flag',\
                  'FAELLT_AUS_TF':'missing_ride_flag','BPUIC':'stop_id','HALTESTELLEN_NAME':'stop_name','ANKUNFTSZEIT':'arrival_time','AN_PROGNOSE':'eta','AN_PROGNOSE_STATUS':'eta_status'\
                   ,'ABFAHRTSZEIT':'departure_time','AB_PROGNOSE':'etd','AB_PROGNOSE_STATUS':'etd_status','DURCHFAHRT_TF':'ride_through_flag'}, inplace=True)

## Test
df_test.rename(columns={'BETRIEBSTAG':'operating_day','FAHRT_BEZEICHNER':'ride_id','BETREIBER_ID':'operator_id','BETREIBER_ABK':'operator_short','BETREIBER_NAME':'operator_name',\
                   'PRODUKT_ID':'transport_mode','LINIEN_ID':'line_id','LINIEN_TEXT':'line_name','UMLAUF_ID':'deviation_id','VERKEHRSMITTEL_TEXT':'product_id','ZUSATZFAHRT_TF':'add_ride_flag',\
                  'FAELLT_AUS_TF':'missing_ride_flag','BPUIC':'stop_id','HALTESTELLEN_NAME':'stop_name','ANKUNFTSZEIT':'arrival_time','AN_PROGNOSE':'eta','AN_PROGNOSE_STATUS':'eta_status'\
                   ,'ABFAHRTSZEIT':'departure_time','AB_PROGNOSE':'etd','AB_PROGNOSE_STATUS':'etd_status','DURCHFAHRT_TF':'ride_through_flag'}, inplace=True)

Index(['operating_day', 'ride_id', 'operator_id', 'operator_short',
       'operator_name', 'transport_mode', 'line_id', 'line_name',
       'deviation_id', 'product_id', 'add_ride_flag', 'missing_ride_flag',
       'stop_id', 'stop_name', 'arrival_time', 'eta', 'eta_status',
       'departure_time', 'etd', 'etd_status', 'ride_through_flag'],
      dtype='object')


**2) update data types** <a name='dtypes'/> | <a href=#index>Back to top</a>

In [42]:
import datetime
## Training
df_train['operating_day'] = pd.to_datetime(df_train['operating_day'], format="%d.%m.%Y")
df_train['arrival_time'] = pd.to_datetime(df_train['arrival_time'], format="%d.%m.%Y %H:%M")
df_train['departure_time'] = pd.to_datetime(df_train['departure_time'], format="%d.%m.%Y %H:%M")
df_train['eta'] = pd.to_datetime(df_train['eta'], format="%d.%m.%Y %H:%M:%S")
df_train['etd'] = pd.to_datetime(df_train['etd'], format="%d.%m.%Y %H:%M:%S")
# integer to string
df_train['stop_id'] = df_train['stop_id'].astype(str)

## Validation
df_valid['operating_day'] = pd.to_datetime(df_valid['operating_day'], format="%d.%m.%Y")
df_valid['arrival_time'] = pd.to_datetime(df_valid['arrival_time'], format="%d.%m.%Y %H:%M")
df_valid['departure_time'] = pd.to_datetime(df_valid['departure_time'], format="%d.%m.%Y %H:%M")
df_valid['eta'] = pd.to_datetime(df_valid['eta'], format="%d.%m.%Y %H:%M:%S")
df_valid['etd'] = pd.to_datetime(df_valid['etd'], format="%d.%m.%Y %H:%M:%S")
# integer to string
df_valid['stop_id'] = df_valid['stop_id'].astype(str)

## Test
df_test['operating_day'] = pd.to_datetime(df_test['operating_day'], format="%d.%m.%Y")
df_test['arrival_time'] = pd.to_datetime(df_test['arrival_time'], format="%d.%m.%Y %H:%M")
df_test['departure_time'] = pd.to_datetime(df_test['departure_time'], format="%d.%m.%Y %H:%M")
df_test['eta'] = pd.to_datetime(df_test['eta'], format="%d.%m.%Y %H:%M:%S")
df_test['etd'] = pd.to_datetime(df_test['etd'], format="%d.%m.%Y %H:%M:%S")
# integer to string
df_test['stop_id'] = df_test['stop_id'].astype(str)

**3) handling NULL values** <a name='null'/> | <a href=#index>Back to top</a>

In [68]:
# stop name
## Training
df_train_merged = df_train.merge(right=df_stops_clean[['stop_name','stop_id']], how='left', on=['stop_id'], suffixes=('','_bis'))
df_train_merged['stop_name'].fillna(df_train_merged['stop_name_bis'], inplace=True)
#print('Even after mapping, we still have {} empty "stop_name"'.format(df_train['stop_name'].isnull().sum()))
df_train_merged['stop_name'].fillna('unknown', inplace=True)

## Validation
df_valid_merged = df_valid.merge(right=df_stops_clean[['stop_name','stop_id']], how='left', on=['stop_id'], suffixes=('','_bis'))
df_valid_merged['stop_name'].fillna(df_valid_merged['stop_name_bis'], inplace=True)
#print('Even after mapping, we still have {} empty "stop_name"'.format(df_train['stop_name'].isnull().sum()))
df_valid_merged['stop_name'].fillna('unknown', inplace=True)

## Test
df_test_merged = df_test.merge(right=df_stops_clean[['stop_name','stop_id']], how='left', on=['stop_id'], suffixes=('','_bis'))
df_test_merged['stop_name'].fillna(df_test_merged['stop_name_bis'], inplace=True)
#print('Even after mapping, we still have {} empty "stop_name"'.format(df_train['stop_name'].isnull().sum()))
df_test_merged['stop_name'].fillna('unknown', inplace=True)

In [44]:
# 3b handling NULL values in product id
## Training
df_train_merged['product_id'].fillna(value='unkown', inplace=True)
## Validation
df_valid_merged['product_id'].fillna(value='unkown', inplace=True)
## Test
df_test_merged['product_id'].fillna(value='unkown', inplace=True)

**4) Feature engineering** <a name='engineer'/> | <a href=#index>Back to top</a>

In [45]:
# 4a create 'start_middle_end' flag
def start_end(row):
    if row['arrival_time'] is pd.NaT:
        return 'start_trip'
    if row['departure_time'] is pd.NaT:
        return 'end_trip'
    return 'middle_stop'

## Training
df_train_merged['start_middle_end'] = df_train_merged.apply(lambda row: start_end(row), axis=1)
## Validation
df_valid_merged['start_middle_end'] = df_valid_merged.apply(lambda row: start_end(row), axis=1)
## Test
df_test_merged['start_middle_end'] = df_test_merged.apply(lambda row: start_end(row), axis=1)

In [46]:
# 4b clean 'ETA/ETD' features
def start_status(row):
    if row['arrival_time'] is pd.NaT:
        return 'NA'
    if row['eta'] is pd.NaT:
        return 'UNKOWN'
    return row['eta_status']

## Training
df_train_merged['eta_status_clean'] = df_train_merged.apply(lambda row: start_status(row), axis=1)
df_train_merged['eta_status_clean'].replace({'GESCHAETZT':'ESTIMATED','UNBEKANNT':'ESTIMATED'}, inplace=True)
## Validation
df_valid_merged['eta_status_clean'] = df_valid_merged.apply(lambda row: start_status(row), axis=1)
df_valid_merged['eta_status_clean'].replace({'GESCHAETZT':'ESTIMATED','UNBEKANNT':'ESTIMATED'}, inplace=True)
## Test
df_test_merged['eta_status_clean'] = df_test_merged.apply(lambda row: start_status(row), axis=1)
df_test_merged['eta_status_clean'].replace({'GESCHAETZT':'ESTIMATED','UNBEKANNT':'ESTIMATED'}, inplace=True)

def end_status(row):
    if row['departure_time'] is pd.NaT:
        return 'NA'
    if row['etd'] is pd.NaT:
        return 'UNKOWN'
    return row['etd_status']

## Training
df_train_merged['etd_status_clean'] = df_train_merged.apply(lambda row: end_status(row), axis=1)
df_train_merged['etd_status_clean'].replace({'GESCHAETZT':'ESTIMATED','UNBEKANNT':'ESTIMATED'}, inplace=True)
## Validation
df_valid_merged['etd_status_clean'] = df_valid_merged.apply(lambda row: end_status(row), axis=1)
df_valid_merged['etd_status_clean'].replace({'GESCHAETZT':'ESTIMATED','UNBEKANNT':'ESTIMATED'}, inplace=True)
## Test
df_test_merged['etd_status_clean'] = df_test_merged.apply(lambda row: end_status(row), axis=1)
df_test_merged['etd_status_clean'].replace({'GESCHAETZT':'ESTIMATED','UNBEKANNT':'ESTIMATED'}, inplace=True)

In [47]:
# 4c time_of_day
## Training
df_train_merged['departure_time_hour'] = df_train_merged['departure_time'].dt.hour
df_train_merged['arrival_time_hour'] = df_train_merged['arrival_time'].dt.hour
df_train_merged['operating_day_of_week'] = df_train_merged['operating_day'].dt.day_name()
## Validation
df_valid_merged['departure_time_hour'] = df_valid_merged['departure_time'].dt.hour
df_valid_merged['arrival_time_hour'] = df_valid_merged['arrival_time'].dt.hour
df_valid_merged['operating_day_of_week'] = df_valid_merged['operating_day'].dt.day_name()
## Test
df_test_merged['departure_time_hour'] = df_test_merged['departure_time'].dt.hour
df_test_merged['arrival_time_hour'] = df_test_merged['arrival_time'].dt.hour
df_test_merged['operating_day_of_week'] = df_test_merged['operating_day'].dt.day_name()

def rush_hour_dep(row):
    if row['departure_time'] is pd.NaT:
        return 'na'
    if row['departure_time_hour'] < 6:
        return 'early_morning'
    if row['departure_time_hour'] < 9:
        return 'peak_morning'
    if row['departure_time_hour'] < 12:
        return 'morning'
    if row['departure_time_hour'] < 14:
        return 'midday'
    if row['departure_time_hour'] < 16:
        return 'afternoon'
    if row['departure_time_hour'] < 20:
        return 'peak_afternoon'
    if row['departure_time_hour'] <= 24:
        return 'evening'
    return 'na'
## Training
df_train_merged['departure_time_of_day'] = df_train_merged.apply(lambda row: rush_hour_dep(row), axis=1)
## Validation
df_valid_merged['departure_time_of_day'] = df_valid_merged.apply(lambda row: rush_hour_dep(row), axis=1)
## Test
df_test_merged['departure_time_of_day'] = df_test_merged.apply(lambda row: rush_hour_dep(row), axis=1)

def rush_hour_arr(row):
    if row['arrival_time'] is pd.NaT:
        return 'na'
    if row['arrival_time_hour'] < 6:
        return 'early_morning'
    if row['arrival_time_hour'] < 9:
        return 'peak_morning'
    if row['arrival_time_hour'] < 12:
        return 'morning'
    if row['arrival_time_hour'] < 14:
        return 'midday'
    if row['arrival_time_hour'] < 16:
        return 'afternoon'
    if row['arrival_time_hour'] < 20:
        return 'peak_afternoon'
    if row['arrival_time_hour'] <= 24:
        return 'evening'
    return 'na'
## Training
df_train_merged['arrival_time_of_day'] = df_train_merged.apply(lambda row: rush_hour_arr(row), axis=1)
## Validation
df_valid_merged['arrival_time_of_day'] = df_valid_merged.apply(lambda row: rush_hour_arr(row), axis=1)
## Test
df_test_merged['arrival_time_of_day'] = df_test_merged.apply(lambda row: rush_hour_arr(row), axis=1)

In [48]:
# 4d delays
## Training
df_train_merged['arrival_delay_min'] = round(((df_train_merged['eta'] - df_train_merged['arrival_time']).dt.total_seconds() / 60),0)
df_train_merged['departure_delay_min'] = round(((df_train_merged['etd'] - df_train_merged['departure_time']).dt.total_seconds() / 60),0)
## Validation
df_valid_merged['arrival_delay_min'] = round(((df_valid_merged['eta'] - df_valid_merged['arrival_time']).dt.total_seconds() / 60),0)
df_valid_merged['departure_delay_min'] = round(((df_valid_merged['etd'] - df_valid_merged['departure_time']).dt.total_seconds() / 60),0)
## Test
df_test_merged['arrival_delay_min'] = round(((df_test_merged['eta'] - df_test_merged['arrival_time']).dt.total_seconds() / 60),0)
df_test_merged['departure_delay_min'] = round(((df_test_merged['etd'] - df_test_merged['departure_time']).dt.total_seconds() / 60),0)

def delay_buckets_dep(row):
    if row['departure_time'] is pd.NaT:
        return 'na'
    if row['missing_ride_flag'] == True: # adding criteria to deal with missing_ride_flag
        return 'important_delay'
    if row['etd'] is pd.NaT:
        return 'unknown'
    if row['departure_delay_min'] >= 5:
        return 'important_delay'
    if row['departure_delay_min'] >= 2:
        return 'delay'
    if row['departure_delay_min'] >= -1:
        return 'on_time'
    return 'early'
## Training
df_train_merged['departure_delay_bucket'] = df_train_merged.apply(lambda row: delay_buckets_dep(row), axis=1)
## Validation
df_valid_merged['departure_delay_bucket'] = df_valid_merged.apply(lambda row: delay_buckets_dep(row), axis=1)
## Test
df_test_merged['departure_delay_bucket'] = df_test_merged.apply(lambda row: delay_buckets_dep(row), axis=1)

def delay_buckets_arr(row):
    if row['arrival_time'] is pd.NaT:
        return 'na'
    if row['missing_ride_flag'] == True: # adding criteria to deal with missing ride_flag
        return 'important_delay'
    if row['eta'] is pd.NaT:
        return 'unknown'
    if row['arrival_delay_min'] >= 5:
        return 'important_delay'
    if row['arrival_delay_min'] >= 2:
        return 'delay'
    if row['arrival_delay_min'] >= -1:
        return 'on_time'
    return 'early'
## Training
df_train_merged['arrival_delay_bucket'] = df_train_merged.apply(lambda row: delay_buckets_arr(row), axis=1)
## Validation
df_valid_merged['arrival_delay_bucket'] = df_valid_merged.apply(lambda row: delay_buckets_arr(row), axis=1)
## Test
df_test_merged['arrival_delay_bucket'] = df_test_merged.apply(lambda row: delay_buckets_arr(row), axis=1)

# plan to work with binary classifiers and therefore regroup target variables in binary classes (e.g.: 'no_delay','delay')
# 'na' won't be in scope
## Training
df_train_merged['arrival_delay_bucket_final'] = df_train_merged['arrival_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','important_delay':'delay'})
df_train_merged['departure_delay_bucket_final'] = df_train_merged['departure_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','important_delay':'delay'})
df_train_merged['arrival_important_delay_bucket_final'] = df_train_merged['arrival_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','delay':'no_delay'})
df_train_merged['departure_important_delay_bucket_final'] = df_train_merged['departure_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','delay':'no_delay'})
## Validation
df_valid_merged['arrival_delay_bucket_final'] = df_valid_merged['arrival_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','important_delay':'delay'})
df_valid_merged['departure_delay_bucket_final'] = df_valid_merged['departure_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','important_delay':'delay'})
df_valid_merged['arrival_important_delay_bucket_final'] = df_valid_merged['arrival_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','delay':'no_delay'})
df_valid_merged['departure_important_delay_bucket_final'] = df_valid_merged['departure_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','delay':'no_delay'})
## Test
df_test_merged['arrival_delay_bucket_final'] = df_test_merged['arrival_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','important_delay':'delay'})
df_test_merged['departure_delay_bucket_final'] = df_test_merged['departure_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','important_delay':'delay'})
df_test_merged['arrival_important_delay_bucket_final'] = df_test_merged['arrival_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','delay':'no_delay'})
df_test_merged['departure_important_delay_bucket_final'] = df_test_merged['departure_delay_bucket'].replace({'early':'no_delay','on_time':'no_delay','unknown':'no_delay','delay':'no_delay'})

**5) Final data sets** <a name='final'/> | <a href=#index>Back to top</a>

In [49]:
# 5a filtering-in only relevant features
# could be using parameter here!
## Training
df_merged_train_clean = df_train_merged[['operating_day','operating_day_of_week','ride_id','operator_short','operator_name','transport_mode','line_id','line_name','deviation_id','product_id','add_ride_flag','missing_ride_flag','stop_id','stop_name',\
          'arrival_time','eta','eta_status_clean','arrival_time_of_day','arrival_delay_min','arrival_delay_bucket',\
            'departure_time','etd','etd_status_clean','departure_time_of_day','departure_delay_min','departure_delay_bucket','start_middle_end',\
            'arrival_delay_bucket_final','departure_delay_bucket_final','arrival_important_delay_bucket_final','departure_important_delay_bucket_final']]
## Validation
df_merged_valid_clean = df_valid_merged[['operating_day','operating_day_of_week','ride_id','operator_short','operator_name','transport_mode','line_id','line_name','deviation_id','product_id','add_ride_flag','missing_ride_flag','stop_id','stop_name',\
          'arrival_time','eta','eta_status_clean','arrival_time_of_day','arrival_delay_min','arrival_delay_bucket',\
            'departure_time','etd','etd_status_clean','departure_time_of_day','departure_delay_min','departure_delay_bucket','start_middle_end',\
            'arrival_delay_bucket_final','departure_delay_bucket_final','arrival_important_delay_bucket_final','departure_important_delay_bucket_final']]
## Test
df_merged_test_clean = df_test_merged[['operating_day','operating_day_of_week','ride_id','operator_short','operator_name','transport_mode','line_id','line_name','deviation_id','product_id','add_ride_flag','missing_ride_flag','stop_id','stop_name',\
          'arrival_time','eta','eta_status_clean','arrival_time_of_day','arrival_delay_min','arrival_delay_bucket',\
            'departure_time','etd','etd_status_clean','departure_time_of_day','departure_delay_min','departure_delay_bucket','start_middle_end',\
            'arrival_delay_bucket_final','departure_delay_bucket_final','arrival_important_delay_bucket_final','departure_important_delay_bucket_final']]

In [50]:
# 5b merging both final dataframes
## Training
df_tr_clean = pd.merge(left=df_merged_train_clean, right=df_stops_clean[['stop_id','transport_mode','provider_short','city','canton','longitude_ch','latitude_ch','altitude']], how='left', on='stop_id', suffixes=('','_stop'))
## Validation
df_va_clean = pd.merge(left=df_merged_valid_clean, right=df_stops_clean[['stop_id','transport_mode','provider_short','city','canton','longitude_ch','latitude_ch','altitude']], how='left', on='stop_id', suffixes=('','_stop'))
## Test
df_te_clean = pd.merge(left=df_merged_test_clean, right=df_stops_clean[['stop_id','transport_mode','provider_short','city','canton','longitude_ch','latitude_ch','altitude']], how='left', on='stop_id', suffixes=('','_stop'))

In [69]:
# 5c splitting final data sets
# arrival set
keep_input_arr = ['operating_day_of_week','operator_short','line_name','product_id','stop_id','arrival_time_of_day','start_middle_end','provider_short','city','canton','longitude_ch','latitude_ch','altitude','transport_mode_stop']
keep_input_cat_arr = ['operating_day_of_week','operator_short','line_name','product_id','stop_id','arrival_time_of_day','start_middle_end','provider_short','city','canton','transport_mode_stop']
keep_input_num_arr = ['longitude_ch','latitude_ch','altitude']
keep_output_arr = ['arrival_delay_bucket_final','arrival_important_delay_bucket_final']

## Training
df_tr_arr = df_tr_clean[~(df_tr_clean['start_middle_end'] == 'start_trip')][(keep_input_arr+keep_output_arr)]
## Validation
df_va_arr = df_va_clean[~(df_va_clean['start_middle_end'] == 'start_trip')][(keep_input_arr+keep_output_arr)]
## Test
df_te_arr = df_te_clean[~(df_te_clean['start_middle_end'] == 'start_trip')][(keep_input_arr+keep_output_arr)]

# departure set
keep_input_dep = ['operating_day_of_week','operator_short','line_name','product_id','stop_id','departure_time_of_day','start_middle_end','provider_short','city','canton','longitude_ch','latitude_ch','altitude','transport_mode_stop']
keep_input_cat_dep = ['operating_day_of_week','operator_short','line_name','product_id','stop_id','departure_time_of_day','start_middle_end','provider_short','city','canton','transport_mode_stop']
keep_input_num_dep = ['longitude_ch','latitude_ch','altitude']
keep_output_dep = ['departure_delay_bucket_final','departure_important_delay_bucket_final']

## Training
df_tr_dep = df_tr_clean[~(df_tr_clean['start_middle_end'] == 'end_trip')][(keep_input_dep+keep_output_dep)]
## Validation
df_va_dep = df_va_clean[~(df_va_clean['start_middle_end'] == 'end_trip')][(keep_input_dep+keep_output_dep)]
## Test
df_te_dep = df_te_clean[~(df_te_clean['start_middle_end'] == 'end_trip')][(keep_input_dep+keep_output_dep)]

In [52]:
print('Training arrival set shape:',df_tr_arr.shape)
print('Training departure set shape:',df_tr_arr.shape)
print('Validation arrival set shape:',df_va_arr.shape)
print('Validation departure set shape:',df_va_arr.shape)
print('Test arrival set shape:',df_te_arr.shape)
print('Test departure set shape:',df_te_arr.shape)

Training arrival set shape: (1948998, 16)
Training departure set shape: (1948998, 16)
Validation arrival set shape: (974789, 16)
Validation departure set shape: (974789, 16)
Test arrival set shape: (978361, 16)
Test departure set shape: (978361, 16)


In [53]:
df_tr_arr.head()

,operating_day_of_week,operator_short,line_name,product_id,stop_id,arrival_time_of_day,start_middle_end,provider_short,city,canton,longitude_ch,latitude_ch,altitude,transport_mode_stop,arrival_delay_bucket_final,arrival_important_delay_bucket_final
0,Thursday,DB,RE,RE,8500090,peak_morning,end_trip,DICH,Basel,BS,2612665,1268525,263,Zug,no_delay,no_delay
1,Thursday,DB,RE,RE,8500090,peak_morning,end_trip,DICH,Basel,BS,2612665,1268525,263,Zug,no_delay,no_delay
2,Thursday,DB,RE,RE,8500090,morning,end_trip,DICH,Basel,BS,2612665,1268525,263,Zug,no_delay,no_delay
4,Thursday,DB,RE,RE,8500090,morning,end_trip,DICH,Basel,BS,2612665,1268525,263,Zug,delay,no_delay
6,Thursday,DB,RE,RE,8500090,morning,end_trip,DICH,Basel,BS,2612665,1268525,263,Zug,delay,important_delay


In [54]:
df_tr_dep.head()

,operating_day_of_week,operator_short,line_name,product_id,stop_id,departure_time_of_day,start_middle_end,provider_short,city,canton,longitude_ch,latitude_ch,altitude,transport_mode_stop,departure_delay_bucket_final,departure_important_delay_bucket_final
3,Thursday,DB,RE,RE,8500090,peak_morning,start_trip,DICH,Basel,BS,2612665,1268525,263,Zug,no_delay,no_delay
5,Thursday,DB,RE,RE,8500090,peak_morning,start_trip,DICH,Basel,BS,2612665,1268525,263,Zug,delay,important_delay
7,Thursday,DB,RE,RE,8500090,peak_morning,start_trip,DICH,Basel,BS,2612665,1268525,263,Zug,no_delay,no_delay
8,Thursday,DB,RE,RE,8500090,morning,start_trip,DICH,Basel,BS,2612665,1268525,263,Zug,delay,no_delay
10,Thursday,DB,RE,RE,8500090,morning,start_trip,DICH,Basel,BS,2612665,1268525,263,Zug,no_delay,no_delay


**6) Data transformation to feed models** <a name='trans'/> | <a href=#index>Back to top</a>

**Input features**

In [55]:
# arrival set
canton = 'VD' # choose which canton to focus on
df_tr_arr_cant = df_tr_arr[df_tr_arr['canton'] == canton]
df_va_arr_cant = df_va_arr[df_va_arr['canton'] == canton]
df_te_arr_cant = df_te_arr[df_te_arr['canton'] == canton]

# departure set
df_tr_dep_cant = df_tr_dep[df_tr_dep['canton'] == canton]
df_va_dep_cant = df_va_dep[df_va_dep['canton'] == canton]
df_te_dep_cant = df_te_dep[df_te_dep['canton'] == canton]

In [56]:
print('Shape training arrival:',df_tr_arr_cant.shape)
print('Shape validation arrival:',df_va_arr_cant.shape)
print('Shape test arrival:',df_te_arr_cant.shape)
print('***')
print('Shape training departure:',df_tr_dep_cant.shape)
print('Shape validation departure:',df_va_dep_cant.shape)
print('Shape test departure:',df_te_dep_cant.shape)

Shape training arrival: (215050, 16)
Shape validation arrival: (108720, 16)
Shape test arrival: (107366, 16)
***
Shape training departure: (215023, 16)
Shape validation departure: (108689, 16)
Shape test departure: (107323, 16)


##### Categorical features

In [57]:
# arrival set
## Training
df_tr_arr_cat = pd.get_dummies(df_tr_arr_cant[keep_input_arr], columns=keep_input_cat_arr)
print('Shape training arrival (cat):',df_tr_arr_cat.shape)
## Validation
df_va_arr_cat = pd.get_dummies(df_va_arr_cant[keep_input_arr], columns=keep_input_cat_arr)
print('Shape validation arrival (cat):',df_va_arr_cat.shape)
## Test
df_te_arr_cat = pd.get_dummies(df_te_arr_cant[keep_input_arr], columns=keep_input_cat_arr)
print('Shape test arrival (cat):',df_te_arr_cat.shape)

print('***')
# departure set
## Training
df_tr_dep_cat = pd.get_dummies(df_tr_dep_cant[keep_input_dep], columns=keep_input_cat_dep)
print('Shape training departure (cat):',df_tr_dep_cat.shape)
## Validation
df_va_dep_cat = pd.get_dummies(df_va_dep_cant[keep_input_dep], columns=keep_input_cat_dep)
print('Shape validation departure (cat):',df_va_dep_cat.shape)
## Test
df_te_dep_cat = pd.get_dummies(df_te_dep_cant[keep_input_dep], columns=keep_input_cat_dep)
print('Shape test departure (cat):',df_te_dep_cat.shape)

Shape training arrival (cat): (215050, 434)
Shape validation arrival (cat): (108720, 424)
Shape test arrival (cat): (107366, 424)
***
Shape training departure (cat): (215023, 434)
Shape validation departure (cat): (108689, 424)
Shape test departure (cat): (107323, 424)


##### Numerical features

In [58]:
from sklearn.preprocessing import StandardScaler

In [59]:
# arrival set
scaler_arr = StandardScaler()
## Training
num_tr_arr = pd.DataFrame(scaler_arr.fit_transform(X=df_tr_arr_cant[keep_input_num_arr]), columns=keep_input_num_arr)
print('Shape training arrival (num):',num_tr_arr.shape)
## Validation
num_va_arr = pd.DataFrame(scaler_arr.transform(X=df_va_arr_cant[keep_input_num_arr]), columns=keep_input_num_arr)
print('Shape validation arrival (num):',num_va_arr.shape)
## Test
num_te_arr = pd.DataFrame(scaler_arr.transform(X=df_te_arr_cant[keep_input_num_arr]), columns=keep_input_num_arr)
print('Shape test arrival (num):',num_te_arr.shape)
print('***')
# departure set
scaler_dep = StandardScaler()
## Training
num_tr_dep = pd.DataFrame(scaler_dep.fit_transform(X=df_tr_dep_cant[keep_input_num_dep]), columns=keep_input_num_dep)
print('Shape training departure (num):',num_tr_dep.shape)
## Validation
num_va_dep = pd.DataFrame(scaler_dep.transform(X=df_va_dep_cant[keep_input_num_dep]), columns=keep_input_num_dep)
print('Shape validation departure (num):',num_va_dep.shape)
## Test
num_te_dep = pd.DataFrame(scaler_dep.transform(X=df_te_dep_cant[keep_input_num_dep]), columns=keep_input_num_dep)
print('Shape test departure (num):',num_te_dep.shape)

Shape training arrival (num): (215050, 3)
Shape validation arrival (num): (108720, 3)
Shape test arrival (num): (107366, 3)
***
Shape training departure (num): (215023, 3)
Shape validation departure (num): (108689, 3)
Shape test departure (num): (107323, 3)


##### Merge categorical and numerical features

In [60]:
# arrival set
## Training
df_tr_arr_raw = df_tr_arr_cat.drop(columns=keep_input_num_arr, axis=1).reset_index(drop=True)
df_tr_arr_input = df_tr_arr_raw.merge(right=num_tr_arr, how='left', left_index=True, right_index=True)
X_tr_arr = df_tr_arr_input.values
print('Shape training input arrival:',X_tr_arr.shape)

## Validation
df_va_arr_raw = df_va_arr_cat.drop(columns=keep_input_num_arr, axis=1).reset_index(drop=True)
df_va_arr_input = df_va_arr_raw.merge(right=num_va_arr, how='left', left_index=True, right_index=True)
X_va_arr = df_va_arr_input.values
print('Shape validation input arrival:',X_va_arr.shape)

## Validation
df_te_arr_raw = df_te_arr_cat.drop(columns=keep_input_num_arr, axis=1).reset_index(drop=True)
df_te_arr_input = df_te_arr_raw.merge(right=num_te_arr, how='left', left_index=True, right_index=True)
X_te_arr = df_te_arr_input.values
print('Shape test input arrival:',X_te_arr.shape)
print('***')

# departure set
## Training
df_tr_dep_raw = df_tr_dep_cat.drop(columns=keep_input_num_dep, axis=1).reset_index(drop=True)
df_tr_dep_input = df_tr_dep_raw.merge(right=num_tr_dep, how='left', left_index=True, right_index=True)
X_tr_dep = df_tr_dep_input.values
print('Shape training input departure:',X_tr_dep.shape)

## Validation
df_va_dep_raw = df_va_dep_cat.drop(columns=keep_input_num_dep, axis=1).reset_index(drop=True)
df_va_dep_input = df_va_dep_raw.merge(right=num_va_dep, how='left', left_index=True, right_index=True)
X_va_dep = df_va_dep_input.values
print('Shape validation input departure:',X_va_dep.shape)

## Validation
df_te_dep_raw = df_te_dep_cat.drop(columns=keep_input_num_dep, axis=1).reset_index(drop=True)
df_te_dep_input = df_te_dep_raw.merge(right=num_te_dep, how='left', left_index=True, right_index=True)
X_te_dep = df_te_dep_input.values
print('Shape test input arrival:',X_te_dep.shape)

Shape training input arrival: (215050, 434)
Shape validation input arrival: (108720, 424)
Shape test input arrival: (107366, 424)
***
Shape training input departure: (215023, 434)
Shape validation input departure: (108689, 424)
Shape test input arrival: (107323, 424)


In [61]:
# arrival set
train_list = df_tr_arr_input.columns.tolist()
valid_list = df_va_arr_input.columns.tolist()
test_list = df_te_arr_input.columns.tolist()

# identify columns which are in train set but not in valid or test sets
for x in train_list:
    if x in test_list: continue
    else:
        print(x)

line_name_AG
line_name_EXT
line_name_EXTL4
product_id_AG
product_id_EXT
stop_id_8501055
stop_id_8501060
stop_id_8501061
stop_id_8501401
stop_id_8519077


In [62]:
# arrival set
df_va_arr_input_reindex = df_va_arr_input.reindex(columns=df_tr_arr_input.columns,fill_value=0.0)
X_va_arr_reindex = df_va_arr_input_reindex.values
print('Reindex shape validation input arrival:',X_va_arr_reindex.shape)
df_te_arr_input_reindex = df_te_arr_input.reindex(columns=df_tr_arr_input.columns,fill_value=0.0)
X_te_arr_reindex = df_te_arr_input_reindex.values
print('Reindex shape test input arrival:',X_te_arr_reindex.shape)
print('***')

# departure set
df_va_dep_input_reindex = df_va_dep_input.reindex(columns=df_tr_dep_input.columns,fill_value=0.0)
X_va_dep_reindex = df_va_dep_input_reindex.values
print('Reindex shape validation input departure:',X_va_dep_reindex.shape)
df_te_dep_input_reindex = df_te_dep_input.reindex(columns=df_tr_dep_input.columns,fill_value=0.0)
X_te_dep_reindex = df_te_dep_input_reindex.values
print('Reindex shape test input departure:',X_te_dep_reindex.shape)

Reindex shape validation input arrival: (108720, 434)
Reindex shape test input arrival: (107366, 434)
***
Reindex shape validation input departure: (108689, 434)
Reindex shape test input departure: (107323, 434)


**Output features**

In [63]:
# arrival set
## Training
y_tr_arr_delay = df_tr_arr_cant['arrival_delay_bucket_final'].values
y_tr_arr_imp_delay = df_tr_arr_cant['arrival_important_delay_bucket_final'].values
print('Shape training output delay arrival:',y_tr_arr_delay.shape)
print('Shape training output important delay arrival:',y_tr_arr_imp_delay.shape)

## Validation
y_va_arr_delay = df_va_arr_cant['arrival_delay_bucket_final'].values
y_va_arr_imp_delay = df_va_arr_cant['arrival_important_delay_bucket_final'].values
print('Shape validation output delay arrival:',y_va_arr_delay.shape)
print('Shape validation output important delay arrival:',y_va_arr_imp_delay.shape)

## Test
y_te_arr_delay = df_te_arr_cant['arrival_delay_bucket_final'].values
y_te_arr_imp_delay = df_te_arr_cant['arrival_important_delay_bucket_final'].values
print('Shape test output delay arrival:',y_te_arr_delay.shape)
print('Shape test output important delay arrival:',y_te_arr_imp_delay.shape)
print('***')

# departure set
## Training
y_tr_dep_delay = df_tr_dep_cant['departure_delay_bucket_final'].values
y_tr_dep_imp_delay = df_tr_dep_cant['departure_important_delay_bucket_final'].values
print('Shape training output delay departure:',y_tr_dep_delay.shape)
print('Shape training output important delay departure:',y_tr_dep_imp_delay.shape)

## Validation
y_va_dep_delay = df_va_dep_cant['departure_delay_bucket_final'].values
y_va_dep_imp_delay = df_va_dep_cant['departure_important_delay_bucket_final'].values
print('Shape validation output delay departure:',y_va_dep_delay.shape)
print('Shape validation output important delay departure:',y_va_dep_imp_delay.shape)

## Test
y_te_dep_delay = df_te_dep_cant['departure_delay_bucket_final'].values
y_te_dep_imp_delay = df_te_dep_cant['departure_important_delay_bucket_final'].values
print('Shape test output delay departure:',y_te_dep_delay.shape)
print('Shape test output important delay departure:',y_te_dep_imp_delay.shape)

Shape training output delay arrival: (215050,)
Shape training output important delay arrival: (215050,)
Shape validation output delay arrival: (108720,)
Shape validation output important delay arrival: (108720,)
Shape test output delay arrival: (107366,)
Shape test output important delay arrival: (107366,)
***
Shape training output delay departure: (215023,)
Shape training output important delay departure: (215023,)
Shape validation output delay departure: (108689,)
Shape validation output important delay departure: (108689,)
Shape test output delay departure: (107323,)
Shape test output important delay departure: (107323,)


**7) Saving input and output features** <a name='save'/> | <a href=#index>Back to top</a>

In [64]:
# Save them into a .npz file
np.savez('capstone_inputoutput.npz',
         # arrival set
         df_tr_arr_input=df_tr_arr_input, # to get columns
         df_tr_arr_input_columns=df_tr_arr_input.columns,
         
         X_tr_arr=X_tr_arr,
         X_va_arr_reindex=X_va_arr_reindex,
         X_te_arr_reindex=X_te_arr_reindex,
         
         y_tr_arr_delay=y_tr_arr_delay,
         y_tr_arr_imp_delay=y_tr_arr_imp_delay,
         
         y_va_arr_delay=y_va_arr_delay,
         y_va_arr_imp_delay=y_va_arr_imp_delay,
         
         y_te_arr_delay=y_te_arr_delay,
         y_te_arr_imp_delay=y_te_arr_imp_delay,
         # departure set
         X_tr_dep=X_tr_dep,
         X_va_dep_reindex=X_va_dep_reindex,
         X_te_dep_reindex=X_te_dep_reindex,
         
         y_tr_dep_delay=y_tr_dep_delay,
         y_tr_dep_imp_delay=y_tr_dep_imp_delay,
         
         y_va_dep_delay=y_va_dep_delay,
         y_va_dep_imp_delay=y_va_dep_imp_delay,
         
         y_te_dep_delay=y_te_dep_delay,
         y_te_dep_imp_delay=y_te_dep_imp_delay
        )

### LOADING input and output features

In [65]:
# Load the npz file
import numpy as np
with np.load('capstone_inputoutput.npz', allow_pickle=False) as npz_file:
    # It's a dictionary-like object
    print(list(npz_file.keys()))

['df_tr_arr_input', 'df_tr_arr_input_columns', 'X_tr_arr', 'X_va_arr_reindex', 'X_te_arr_reindex', 'y_tr_arr_delay', 'y_tr_arr_imp_delay', 'y_va_arr_delay', 'y_va_arr_imp_delay', 'y_te_arr_delay', 'y_te_arr_imp_delay', 'X_tr_dep', 'X_va_dep_reindex', 'X_te_dep_reindex', 'y_tr_dep_delay', 'y_tr_dep_imp_delay', 'y_va_dep_delay', 'y_va_dep_imp_delay', 'y_te_dep_delay', 'y_te_dep_imp_delay']


In [66]:
import numpy as np
with np.load('capstone_inputoutput.npz', allow_pickle=True) as npz_file:
    # Load the arrays
    # arrival set
    df_tr_arr_input=npz_file['df_tr_arr_input']
    df_tr_arr_input_columns=npz_file['df_tr_arr_input_columns']
    
    X_tr_arr = npz_file['X_tr_arr']
    X_va_arr_reindex = npz_file['X_va_arr_reindex']
    X_te_arr_reindex = npz_file['X_te_arr_reindex']
    
    y_tr_arr_delay = npz_file['y_tr_arr_delay']
    y_tr_arr_imp_delay = npz_file['y_tr_arr_imp_delay']
    y_va_arr_delay = npz_file['y_va_arr_delay']
    y_va_arr_imp_delay = npz_file['y_va_arr_imp_delay']
    y_te_arr_delay = npz_file['y_te_arr_delay']
    y_te_arr_imp_delay = npz_file['y_te_arr_imp_delay']
    # departure set
    X_tr_dep = npz_file['X_tr_dep']
    X_va_dep_reindex = npz_file['X_va_dep_reindex']
    X_te_dep_reindex = npz_file['X_te_dep_reindex']
    
    y_tr_dep_delay = npz_file['y_tr_dep_delay']
    y_tr_dep_imp_delay = npz_file['y_tr_dep_imp_delay']
    y_va_dep_delay = npz_file['y_va_dep_delay']
    y_va_dep_imp_delay = npz_file['y_va_dep_imp_delay']
    y_te_dep_delay = npz_file['y_te_dep_delay']
    y_te_dep_imp_delay = npz_file['y_te_dep_imp_delay']

print('df_merged_final:', X_tr_arr.shape)

df_merged_final: (215050, 434)


In [67]:
df_tr_arr_input_columns

array(['operating_day_of_week_Friday', 'operating_day_of_week_Monday',
       'operating_day_of_week_Saturday', 'operating_day_of_week_Sunday',
       'operating_day_of_week_Thursday', 'operating_day_of_week_Tuesday',
       'operating_day_of_week_Wednesday', 'operator_short_BC-cmBC',
       'operator_short_BLS-bls', 'operator_short_LEB',
       'operator_short_MBC', 'operator_short_MOB',
       'operator_short_MVR-cev', 'operator_short_NStCM',
       'operator_short_SBB', 'operator_short_TPC-al',
       'operator_short_TPF', 'operator_short_TRAVYS-pbr',
       'operator_short_TRAVYS-ysc', 'line_name_AG', 'line_name_EC',
       'line_name_EXT', 'line_name_EXTL4', 'line_name_IC',
       'line_name_IC1', 'line_name_IC5', 'line_name_IR', 'line_name_IR15',
       'line_name_IR90', 'line_name_PE', 'line_name_R', 'line_name_RE',
       'line_name_S', 'line_name_S1', 'line_name_S2', 'line_name_S3',
       'line_name_S30', 'line_name_S4', 'line_name_S5', 'line_name_S50',
       'line_name_S7',